In [ ]:
%load_ext autoreload
%autoreload 2

### Running time

/Volumes/neuropixel:
    - Alessandro rem-stim2 (56GB): 1h45 (including binfile copying with 20 jobs <20min)

In [ ]:
from pathlib import Path
from joblib import Parallel, delayed
import itertools

In [ ]:
from ecephys_analyses.data import channel_groups, paths, parameters
from ecephys_analyses.sorting import run_sorting

In [ ]:
import spikeinterface.sorters as ss
ss.installed_sorters()

In [ ]:
## USER

sorting_conditions = [
    'ks2_5_catgt_Th=12-10_lam=50_8s-batches',
]

use_catgt_data = True  # Use catgt-preprocessed data saved at the "catgt" root

##########
## Manual:
# Subject, condition
data_conditions = [
#     ('Alessandro', '8-31-2020_REMmerged_imec0'),
]
##########
## Auto

condition_groups = [
    ('Doppio', 'SD'),
    ('Allan', 'SD'),
    ('Luigi', 'SD'),
]

data_conditions = []
for subject, condition_group in condition_groups:
    data_conditions += [
        (subject, cond)
        for cond in paths.get_conditions(subject, condition_group)
    ]
print(f'Data condition N={len(data_conditions)} (Looks good?): {data_conditions}')

#########

tgt_root_key = 'sleep-homeostasis'  # Where we save sorting. A key in roots.yml

########

bad_channels = None  # TODO
rerun_existing = False  # Ignore if 'spike_times.npy' in output dir
dry_run = True # Create output dirs, don't run sorting
clean_dat_file = True  # Remove `recording.dat` generated by spikeinterface if sorting was successful

n_jobs = 1
assert n_jobs == 1

## end USER

In [ ]:
if n_jobs == 1:
    for (
        (subject, condition),
        sorting_condition
    ) in itertools.product(data_conditions, sorting_conditions):
        run_sorting(
            subject,
            condition,
            sorting_condition,
            catgt_data=use_catgt_data,
            tgt_root_key=tgt_root_key,
            bad_channels=bad_channels,
            rerun_existing=rerun_existing,
            dry_run=dry_run,
            clean_dat_file=clean_dat_file
        )
else:
    
    parallel = Parallel(
        n_jobs=n_jobs,
        backend='multiprocessing',
    )(
        delayed(run_sorting)(
            subject,
            condition,
            sorting_condition,
            catgt_data=use_catgt_data,
            bad_channels=bad_channels,
            rerun_existing=rerun_existing,
            dry_run=dry_run,
            clean_dat_file=clean_dat_file
        ) for ((subject, condition), sorting_condition)
        in itertools.product(data_conditions, sorting_conditions)
    )